In [2]:
import pandas as pd
import os
import zipfile

# 1. Ler Arquivos

In [4]:
# Sobe um nível no diretório
os.chdir("..")

In [5]:
# Armazena o novo diretório de trabalho atual
diretorio_atual = os.getcwd()
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [6]:
# Caminho dos diretorios
dir_preprocessed_II = os.path.join(diretorio_atual, "data_cvm\preprocessed_II")

In [7]:
# Define o diretório
os.chdir(dir_preprocessed_II)

In [8]:
# Ler dados trimestral
df_trim = pd.read_csv("equatorial_balanco_trim.csv")

In [9]:
# Ler dados anual
df_anual = pd.read_csv("equatorial_balanco_anual.csv")

In [10]:
df = pd.concat([df_anual, df_trim], ignore_index=True)

# 2. Análise Exploratória

## 2.1 Verficando se há meses duplicados 

In [11]:
teste = df[(df['CD_CONTA'] == '1') & (df['ANO'] == 2022)]

In [12]:
display(teste)

,CNPJ_CIA,DENOM_CIA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,DT_INI_EXERC,TIPO_DEM,ANO,MES
2225,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-12-31,1,Ativo Total,90617397.0,NaN,Balanço Patrimonial Ativo,2022,12
9100,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-03-31,1,Ativo Total,70429153.0,NaN,Balanço Patrimonial Ativo,2022,3
9185,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-06-30,1,Ativo Total,71409299.0,NaN,Balanço Patrimonial Ativo,2022,6
9270,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-09-30,1,Ativo Total,70907242.0,NaN,Balanço Patrimonial Ativo,2022,9


# 3. Manipulação dos Dados

## 3.1 Deletando as Colunas

In [13]:
df_droped_column_anual = df_anual.drop(
    columns=['DT_FIM_EXERC','DT_INI_EXERC'])

In [14]:
df_droped_column = df.drop(
    columns=['DT_FIM_EXERC','DT_INI_EXERC'])

## 3.2 Inserindo a Coluna Período

### 3.2.1 Anual

In [15]:
df_droped_column_anual['PERIODO'] = 'ANUAL'

### 3.3.2 Trimestral

In [16]:
# Função para inserir a coluns Período
def insere_periodo_trimestral(df):
    df = df.copy()

    # Dicionário para mapear os meses aos trimestres
    mes_para_trimestre = {3: 1, 6: 2, 9: 3, 12: 4}
    
    # Filtrar apenas os anos e meses relevantes
    anos_relevantes = list(range(2011, 2025))
    meses_relevantes = [3, 6, 9, 12]

    # Aplicar a lógica de período trimestral
    df = df[(df['ANO'].isin(anos_relevantes)) & (df['MES'].isin(meses_relevantes))]
    df['PERIODO'] = df.apply(lambda row: f"{mes_para_trimestre[row['MES']]}T{row['ANO']}", axis=1)
    
    return df

In [17]:
# Inserindo a coluna período
df_droped_column = insere_periodo_trimestral(df_droped_column)

In [19]:
teste = df_droped_column[(df_droped_column['CD_CONTA'] == '1') & (df_droped_column['ANO'] == 2023)]
display(teste)

,CNPJ_CIA,DENOM_CIA,CD_CONTA,DS_CONTA,VL_CONTA,TIPO_DEM,ANO,MES,PERIODO
2442,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,1,Ativo Total,103643452.0,Balanço Patrimonial Ativo,2023,12,4T2023
9744,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,1,Ativo Total,92834649.0,Balanço Patrimonial Ativo,2023,3,1T2023
9829,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,1,Ativo Total,95701577.0,Balanço Patrimonial Ativo,2023,6,2T2023
9916,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,1,Ativo Total,98775410.0,Balanço Patrimonial Ativo,2023,9,3T2023


## 3.5 Concatenando Dados Anuais e Trimestrais

In [20]:
df_final = pd.concat([df_droped_column, df_droped_column_anual], ignore_index=True)

In [21]:
df_final.head(3)

,CNPJ_CIA,DENOM_CIA,CD_CONTA,DS_CONTA,VL_CONTA,TIPO_DEM,ANO,MES,PERIODO
0,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,1,Ativo Total,3398831.0,Balanço Patrimonial Ativo,2011,12,4T2011
1,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,1.01,Ativo Circulante,1053617.0,Balanço Patrimonial Ativo,2011,12,4T2011
2,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,1.01.01,Caixa e Equivalentes de Caixa,242655.0,Balanço Patrimonial Ativo,2011,12,4T2011


## 3.6 Alterando o Nome das Colunas

In [22]:
colunas = ['CNPJ','EMPRESA','CONTA','DESCRIÇÃO','VALOR','DEMONSTRATIVO','ANO', 'MES', 'PERIODO']
df_final.columns = colunas

# 4. Exportando os Dados

In [28]:
# Sobe um nível no diretório
os.chdir("..")

In [29]:
diretorio_atual = os.getcwd()
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [30]:
# Define o diretório de dados processados
# Caminho dos diretorios
dir_processed_data = os.path.join(diretorio_atual, "data_cvm\processed")
os.chdir(dir_processed_data)

In [31]:
df_final.to_csv('balanco_patrimonial_completo.csv', index=False)